In [ ]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.0 MB/s eta 0:00:00


In [ ]:
# Instalação e configuração de variaveis ambiente para utilizar Google Drive
# Se utilizar o Colab ajustar para True para instalação dos pre-requisitos
colab = True

if colab==True:
  from google.colab import drive
  drive.mount('/content/drive')

  # Google Drive
  dir_base = "/content/drive/MyDrive/jupyter/pcd_porto_seguro_safe_driver_prediction/data/"
  # raiz
  dir_base_root = "/content/drive/MyDrive/jupyter/pcd_porto_seguro_safe_driver_prediction/"
  import sys
  # caution: path[0] is reserved for script path (or '' in REPL)
  sys.path.append(dir_base_root)

  # Insalar depedências


else:
  # Diretorio da base dos dados
  # Local PC
  dir_base = "data/"


Mounted at /content/drive


In [ ]:
#Importação de Bibliotecas utilizadas no projeto/modelo
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib. pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score, auc
from sklearn.linear_model import LogisticRegression

import pod_academy_functions as pod

-----------------------------------------------------
------------ PoD Academy Functions ------------------
Só Alegria!
Você está usando o pacote de funções da PoD Academy
Turma: 2023
Autor: PoD Academy
Professor: Bruno Jardim
Versão: 1.0.0
Data: 08/10/2023
-----------------------------------------------------


In [ ]:
# Funções
# Métricas do modelo

# Métricas do modelo

def calculate_metrics(nm_modelo, model, X_train, y_train, X_test, y_test):

    # Fazendo predições
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculando as métricas para o conjunto de treino
    accuracy_train = accuracy_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred)
    recall_train = recall_score(y_train, y_train_pred)
    auc_roc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])

    # Calculando o Índice Gini e Estatística KS para o conjunto de treino
    probabilities_train = model.predict_proba(X_train)[:, 1]
    df_train = pd.DataFrame({'true_labels': y_train, 'predicted_probs': probabilities_train})
    df_train = df_train.sort_values(by='predicted_probs', ascending=False)
    df_train['cumulative_true'] = df_train['true_labels'].cumsum() / df_train['true_labels'].sum()
    df_train['cumulative_false'] = (1 - df_train['true_labels']).cumsum() / (1 - df_train['true_labels']).sum()
    ks_statistic_train = max(abs(df_train['cumulative_true'] - df_train['cumulative_false']))
    gini_index_train = 2 * auc_roc_train - 1

    # Calculando Gini Normalizado - Treino
    gini_index_train_n = pod.gini_normalizado(y_train, probabilities_train)


    # Calculando as métricas para o conjunto de teste
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    auc_roc_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

    # Calculando o Índice Gini e Estatística KS para o conjunto de teste
    probabilities_test = model.predict_proba(X_test)[:, 1]
    df_test = pd.DataFrame({'true_labels': y_test, 'predicted_probs': probabilities_test})
    df_test = df_test.sort_values(by='predicted_probs', ascending=False)
    df_test['cumulative_true'] = df_test['true_labels'].cumsum() / df_test['true_labels'].sum()
    df_test['cumulative_false'] = (1 - df_test['true_labels']).cumsum() / (1 - df_test['true_labels']).sum()
    ks_statistic_test = max(abs(df_test['cumulative_true'] - df_test['cumulative_false']))
    gini_index_test = 2 * auc_roc_test - 1

    # Calculando Gini Normalizado - Treino
    gini_index_test_n = pod.gini_normalizado(y_test, probabilities_test)

    # Criando o DataFrame com as métricas calculadas
    metrics_df = pd.DataFrame({
        'Algoritmo': [nm_modelo, nm_modelo],
        'Conjunto': ['Treino', 'Teste'],
        'Acuracia': [accuracy_train, accuracy_test],
        'Precisao': [precision_train, precision_test],
        'Recall': [recall_train, recall_test],
        'AUC_ROC': [auc_roc_train, auc_roc_test],
        'GINI': [gini_index_train, gini_index_test],
        'GINI_N': [gini_index_train_n, gini_index_test_n],
        #'PROBA': [probabilities_train, probabilities_test],
        'KS': [ks_statistic_train, ks_statistic_test]
    })
    return metrics_df

In [ ]:
df_train = pd.read_csv(dir_base + 'abt_train.csv')
df_test = pd.read_csv(dir_base + 'abt_test.csv')

In [ ]:
df_train.head(2)

,Unnamed: 0,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1,id,target
0,510263,0.553925,-0.543007,1.321585,-0.311952,-0.805988,-0.587013,2.256576,-0.47767,-0.019446,...,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,1274939,0
1,566227,-0.957891,-0.543007,0.211296,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,...,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,1415654,0


In [ ]:
df_train.columns

Index(['Unnamed: 0', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_04_cat', 'ps_car_06_cat', 'ps_car_09_cat',
       'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13',
       'ps_car_14', 'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03',
       'ps_calc_04', 'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08',
       'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13',
       'ps_calc_14', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin',
       'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin',
       'ps_car_05_cat_0.5261814898236643', 'ps_car_05_cat_1.0',
       'ps_car_07_cat_0.9479847665595149', 'ps_car_07_c

In [ ]:
X = df_train.drop(columns=['id', 'target', 'Unnamed: 0'])
y = df_train['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6)

In [ ]:
list_columns = X_train.columns

In [ ]:
# Testar outros modelos
# desativei o RandomForestClassifier( random_state=6),
algorithms = [DecisionTreeClassifier(criterion='gini', random_state=6),
             LogisticRegression(solver='liblinear',  random_state=6),
             GradientBoostingClassifier( random_state=6),
             XGBClassifier( random_state=6),
             lgb.LGBMClassifier( random_state=6)]

for algorithm in algorithms:
    name_algorithm = str(algorithm)[:str(algorithm).find("(")]
    #Treino no momdelo
    algorithm.fit(X_train, y_train)

    # Avaliar modelo
    metrics = calculate_metrics(name_algorithm, algorithm, X_train, y_train, X_test, y_test)
    display(metrics)

,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,GINI_N,KS
0,DecisionTreeClassifier,Treino,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000
1,DecisionTreeClassifier,Teste,0.918414,0.047575,0.066701,0.508444,0.016888,0.020647,0.01921


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,GINI_N,KS
0,LogisticRegression,Treino,0.963710,0.375,0.000331,0.619977,0.239953,0.239953,0.172400
1,LogisticRegression,Teste,0.964037,0.500,0.000519,0.625963,0.251925,0.251925,0.186821


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,GINI_N,KS
0,GradientBoostingClassifier,Treino,0.963782,0.900,0.001985,0.651494,0.302988,0.302988,0.211104
1,GradientBoostingClassifier,Teste,0.963981,0.125,0.000260,0.635956,0.271911,0.271911,0.199497


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,GINI_N,KS
0,XGBClassifier,Treino,0.964510,0.990099,0.022051,0.856714,0.713427,0.713427,0.541728
1,XGBClassifier,Teste,0.963879,0.052632,0.000260,0.598215,0.196429,0.196429,0.149602


[LightGBM] [Info] Number of positive: 9070, number of negative: 240918
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1424
[LightGBM] [Info] Number of data points in the train set: 249988, number of used features: 59
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036282 -> initscore=-3.279484
[LightGBM] [Info] Start training from score -3.279484


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,GINI_N,KS
0,LGBMClassifier,Treino,0.963806,1.0,0.002426,0.755398,0.510796,0.510796,0.376738
1,LGBMClassifier,Teste,0.964000,0.0,0.000000,0.632223,0.264446,0.264446,0.191279


In [ ]:
algorithms = [LogisticRegression(solver='liblinear',  random_state=6)]

for algorithm in algorithms:

    name_algorithm = str(algorithm)[:str(algorithm).find("(")]
    # Treino do modelo
    algorithm.fit(X_train,y_train)

    # Avaliar modelo
    metrics = calculate_metrics(name_algorithm,algorithm, X_train, y_train, X_test, y_test)
    display(metrics)

,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,GINI_N,KS
0,LogisticRegression,Treino,0.963710,0.375,0.000331,0.619977,0.239953,0.239953,0.172400
1,LogisticRegression,Teste,0.964037,0.500,0.000519,0.625963,0.251925,0.251925,0.186821


In [ ]:
import pickle

with open(dir_base + 'porto_seguro_modelo_baseline_lgbm_v1.pkl', 'wb') as file:
  pickle.dump(algorithm, file)